# Opulent Voice Numerology

For now, the opv-cxx-demod package supports a single fixed set of parameter choices (one "numerology"). This notebook computes them and documents the process of choosing them.

## Nomenclature
`Opus` is the chosen vocoder; `OPV` is short for ***Opulent Voice*** (see what we did there?) and refers to the overall framing, coding, and modulation scheme.

Following M17, we define the following types of bits:

| Type  | Description |
|-------|-----------------------------------------------|
| type1 | initial data bits |
| type2 | bits after encoding |
| type3 | reserved (encoded bits after puncturing in M17) |
| type4 | decorrelated and interleaved type3 bits |


In [12]:
import math
import sympy

### Vocoder Bit Rate

`opus_bitrate` is the fixed bit rate the vocoder is configured to use. For now, we are operating Opus in CBR (constant bit rate) mode, so it always uses exactly this bit rate. It would also be possible to operate Opus in VBR (variable bit rate) mode, in which case this becomes the maximum allowed bit rate (and any unused bits can be made available for other purposes).
The bit rate is chosen to achieve the desired voice quality. Opus recommends a bit rate of 16k to 20k for "Wideband" audio (8 kHz bandwidth).

In [13]:
opus_bitrate = 16000    # bits/second

### Vocoder Frame Duration

`opus_frame_duration` is the length of the vocoder frame in seconds. This must be chosen from Opus's list of supported frame sizes: 2.5, 5, 10, 20, 40, or 60 ms.

In [14]:
opus_frame_duration = 20 * 0.001
opus_frame_type1_bytes = int(opus_frame_duration * opus_bitrate) / 8
assert opus_frame_type1_bytes == int(opus_frame_type1_bytes)
opus_frame_type1_bytes = int(opus_frame_type1_bytes)
opus_frame_duration, opus_frame_type1_bytes

(0.02, 40)

### Vocoder Frames per Channel Frame

`opus_frames_per_opv_frame` is the number of vocoder frames packed into each stream-mode frame defined for Opulent Voice.

In [15]:
opus_frames_per_opv_frame = 2
opv_frame_duration = opus_frames_per_opv_frame * opus_frame_duration
opv_frame_duration

0.04

### PCM Sample Rate
The input to the voice encoder and the output from the voice decoder are raw streams of 16-bit unsigned integer samples, single channel, at a predetermined sample rate. This rate might vary from implementation to implementation, but 8000 samples/second is a typical value for speech. The rest of the numerology does not depend on the PCM sample rate.

In [16]:
audio_sample_rate = 8000    # 16-bit samples/second
audio_samples_per_frame = int(audio_sample_rate * opus_frame_duration * opus_frames_per_opv_frame)
audio_samples_per_frame

320

### Frame Header
For now, the frame header is a black box of a certain size.

In [17]:
fheader_type1_bytes = 12   # multiples of 3 fill the Golay code
fheader_type1_bits = fheader_type1_bytes * 8
fheader_type2_bits = fheader_type1_bits * 2   # 12,24 Golay codes, rate 1/2
fheader_type3_bits = fheader_type2_bits   # no puncturing for fheader
fheader_type3_bits

192

### Payload (Vocoder Data in Stream Mode)

In [18]:
raw_payload_bits = opus_bitrate * opv_frame_duration    # this is total bits for both Opus frames
payload_type1_bits = raw_payload_bits + 4   # convolutional encoder tail
payload_type2_bits = payload_type1_bits * 2 # Rate 1/2 convolutional code

# We don't use any puncturing. Type 3 is same as Type 2.
payload_type3_bits = payload_type2_bits

payload_type3_bits

1288.0

### Combined Type 3 Frame

In [19]:
opv_frame_type3_bits = fheader_type3_bits + payload_type3_bits
opv_frame_type3_bytes = opv_frame_type3_bits / 8
assert opv_frame_type3_bits == opv_frame_type3_bytes * 8
opv_frame_type3_bytes

185.0

### Transmitted Symbol Rate for Voice Stream Mode
The combined type3 frame is prefixed with a 16-bit sync word. The transmitted symbol rate is chosen so that the prefixed frames come out at the correct frame rate.

In [20]:
opv_bit_rate = (16 + opv_frame_type3_bits) / opv_frame_duration
opv_symbol_rate = opv_bit_rate / 2  # 4FSK, so 2 bits per symbol
opv_symbol_rate

18700.0

## BERT Mode
To facilitate testing, we have a Bit Error Rate Testing (BERT) mode. A fixed pseudorandom sequence is broken up into frames of the same length as the payload of an OPV voice frame, and inserted into frames in exactly the same way as the voice data.

A prime number is chosen as the number of bits in each frame, so that each frame is unique over a relatively long period of time.

In [26]:
bert_type1_size = int(raw_payload_bits)
bert_preencode_padding = 0
while not sympy.isprime(bert_type1_size):
    bert_type1_size -= 1
    bert_preencode_padding += 1

(bert_type1_size, bert_preencode_padding)

(631, 9)

## Creating Numerology.h
The following can be copied and pasted into source file `Numerology.h` to define everything we've calculated. Names have been kept from the current draft source code, though they are not wholely consistent.

In [22]:
print('''#pragma once

// This file was generated automatically by numerology.ipynb.

#include <array>

namespace mobilinkd
{''')

print(f'    const int opus_bitrate = {opus_bitrate};')
print(f'    const int encoded_fheader_size = {fheader_type3_bits};')
print(f'    const int punctured_payload_size = {payload_type3_bits};')
print(f'    const int frame_size_bits = {opv_frame_type3_bits};')
print(f'    const int frame_size_bytes = {opv_frame_type3_bytes};')
print(f'    const int audio_sample_rate = {audio_sample_rate};')
print(f'    const int audio_frame_size = {audio_samples_per_frame};')
print(f'    const int opus_frame_size_bytes = {opus_frame_type1_bytes};')
# wrong in C++ code! print(f'    const int audio_payload_bits = {};')
print(f'    const int symbol_rate = {opv_symbol_rate};')
print(f'    const int bert_bits_per_frame = {bert_bits_per_frame};')
print(f'    const int bert_extra_bits = {bert_preencode_padding};')
print(f'    const int bert_encoded_size = {bert_type2_bits};')
print(f'    const int bert_punctured_size = {bert_punctured_size};')
print(f'    const int bert_postpuncture_padding = {bert_postpuncture_padding};')

print('''
    using fheader_t = std::array<int8_t, encoded_fheader_size>;   // one bit per int8_t

}''')

#pragma once

// This file was generated automatically by numerology.ipynb.

#include <array>

namespace mobilinkd
{
    const int opus_bitrate = 16000;
    const int encoded_fheader_size = 192;
    const int punctured_payload_size = 1288.0;
    const int frame_size_bits = 1480.0;
    const int frame_size_bytes = 185.0;
    const int audio_sample_rate = 8000;
    const int audio_frame_size = 320;
    const int opus_frame_size_bytes = 40;
    const int symbol_rate = 18700.0;
    const int bert_bits_per_frame = 797;
    const int bert_extra_bits = 6;
    const int bert_encoded_size = 1600;
    const int bert_punctured_size = 1467;
    const int bert_postpuncture_padding = 13.0;

    using fheader_t = std::array<int8_t, encoded_fheader_size>;   // one bit per int8_t

}
